In [52]:
#valid test cases
Vexps= [
    '(4**5)',
    '(4-+5)',
    '(5+-1)',
    '(3/1.5)',
    '(4/-1)',
    '(4*-2)',
    '((4+(5-4))*2)',
    '(3.14**2)',
    '(-2+-2)'
]
#invalid test cases
IVexps= [
    '((4+5))',
    '(4+*5)',
    '(4/(1-1))',
    '(4/*5)',
    '(4***5)',
    '(2 2 * 4)'
]

#invalid tokens

<h1>Tokenisation</h1>

In [53]:
def tokenize_equation(equation):
    tokens = []
    number = ""
    i = 0  # Position index

    while i < len(equation):
        char = equation[i]

        if char.isdigit() or char == '.':  # Build multi-digit numbers & decimals
            number += char

        else:
            if number:  # Store the completed number
                tokens.append(number)
                number = ""

            # Check for '**' operator
            if char == '*' and i + 1 < len(equation) and equation[i + 1] == '*':
                tokens.append('**')
                i += 1  # Skip the next '*'

            elif char in "+-*/^()":  # Other operators and parentheses
                tokens.append(char)
                
        i += 1  # Move to next character
        
    output = []

    i = 0
    while i < len(tokens):
        # Check if the current element is a negative sign and the previous one is an operator
        if tokens[i] in ['-', '+'] and i - 1 >= 0 and tokens[i - 1] in ['+', '-', '*', '/', '(', '**']:
            output.append(tokens[i] + tokens[i + 1])  # Merge '-' and the next number
            i += 2  # Skip the next element, since it's already merged
        else:
            output.append(tokens[i])
            i += 1

    return output

In [54]:
tokens = tokenize_equation(IVexps[-1])
print(tokens)

['(', '2', '2', '*', '4', ')']


In [55]:
exp1 = '((-500+(4*3.14))/(2**3))'
tokens = tokenize_equation(exp1)
print(tokens)

['(', '(', '-500', '+', '(', '4', '*', '3.14', ')', ')', '/', '(', '2', '**', '3', ')', ')']


<h1>Validation for brackets</h1>

In [56]:
IVbrackets = [
    ['(', '5', '**', '2', ')', '-', '1'], # no open and close bracket
    ['(', '6', '+', '(', '2', '*', '3', ')', ')', '/', '0'], # no open and close brackets
    ['(', '4', '-', '2', '**', '3', ')'], # missing inner bracket
    ['(', '(', '9', '-', '(', '6', '/', '0', ')', ')', ')'], #extra double brackets
    ['(', '(', '3', '*', '4', ')', '+', '5', '/', '(', '2', '-', '1', ')', ')'], #missing inner brackets
    ['(', '3', '+', '4', ')', ')'], #extra single bracket
    ['(', '(', '3', '+', '4', ')', '(', '(', '(', '5', '-', '0', ')', '*', '(', '1',  '**', '3', ')', ')'] #extra single bracket
]

In [57]:
def is_valid_expression(expression):
    stack = []  # Stack to store bracketed expressions
    i = 0
    n = len(expression)
    validation = 0
    while i < n:
        # Make sure everything must be under one bracket first
        if stack == []:
            validation += 1

        # If all of the expression is not under 1 bracket, return False
        if validation > 1:
            return False
        
            
        char = expression[i]
        if char == '(':
            stack.append([])  # Start a new bracketed expression
        elif char == ')':
            if not stack or not stack[-1]:  # Check for misplaced brackets
                return False
            sub_expr = stack.pop()  # Get the last bracketed expression
            if not is_valid_subexpression(sub_expr):  # Validate its contents
                return False
            if stack:
                stack[-1].append('N')  # Represent valid nested expression as 'N'
        elif char.isdigit() or char == '.':  # Check for numbers (including floating points)
            num = char
            decimal_seen = char == '.'  # Track if we see a decimal point
            while i + 1 < n and (expression[i + 1].isdigit() or (expression[i + 1] == '.' and not decimal_seen)):
                i += 1
                if expression[i] == '.':
                    decimal_seen = True
                num += expression[i]
            if num.count('.') > 1:  # More than one decimal in a number is invalid
                return False
            if stack:
                stack[-1].append(num)
        elif char in '+-*/':  # Check for operators
            # Handle exponentiation `**` as a single operator
            if char == '*' and i + 1 < n and expression[i + 1] == '*':
                char = '**'
                i += 1
            # Handle negative numbers
            if char in '-+' and (i == 0 or expression[i - 1] == '(' or expression[i - 1] in '+-*/'):
                num = char
                while i + 1 < n and (expression[i + 1].isdigit() or expression[i + 1] == '.'):
                    i += 1
                    num += expression[i]
                if num.count('.') > 1:  # More than one decimal in a number is invalid
                    return False
                if stack:
                    stack[-1].append(num)
            else:
                if stack:
                    stack[-1].append(char)
        elif char == ' ':
            pass  # Ignore spaces
        else:
            return False  # Invalid character
        i += 1
        print(stack)

    return not stack  # Stack should be empty if all brackets are properly closed

def is_valid_subexpression(sub_expr):
    """
    Checks if a bracketed expression is valid:
    - Either contains exactly [operand, operator, operand]
    - OR contains a valid nested expression.
    """
    if len(sub_expr) == 3 and is_number(sub_expr[0]) and sub_expr[1] in ['+', '-', '*', '/', '**'] and is_number(sub_expr[2]):
        return True
    elif len(sub_expr) == 3 and sub_expr[0] == 'N' and sub_expr[1] in ['+', '-', '*', '/', '**'] and is_number(sub_expr[2]):
        return True
    elif len(sub_expr) == 3 and is_number(sub_expr[0]) and sub_expr[1] in ['+', '-', '*', '/', '**'] and sub_expr[2] == 'N':
        return True
    elif len(sub_expr) == 3 and sub_expr[0] == 'N' and sub_expr[1] in ['+', '-', '*', '/', '**'] and sub_expr[2] == 'N':  
        return True
    return False

def is_number(value):
    """ Helper function to check if a string represents a valid integer or float """
    try:
        float(value)  # Works for both integers and floating point numbers
        return True
    except ValueError:
        return False

In [58]:
test_cases = [
    "(1+2)",          # ✅ Valid
    "(1 + (1 + 2))",    # ✅ Valid
    "(1 + 2 + 3)",      # ❌ Invalid (Extra operator)
    "(1 + (1))",        # ❌ Invalid (Only one operand inside inner brackets)
    "(1 + (2 * 3))",    # ✅ Valid
    "((1 + 2) * 3)",    # ✅ Valid
    "(1 + (1 + (2 * 3)))", # ✅ Valid
    "(1 + (2 ** 5))",   # ✅ Valid (Exponentiation now supported)
    "(3 ** (2 + 1))",   # ✅ Valid (Nested exponentiation)
    "(1 ** 2 ** 3)",    # ❌ Invalid (Only one operator allowed per bracket)
    "(1 + (2 * 3)",     # ❌ Invalid (Missing closing bracket)
    "((-500+(4*3.14))/(2**3))", 
    "((500 + (4*3.14))/2)",
    "((4 + 4))",
    "(4+4)+(4+4)",
    "((4+4)+(4+4))+(5+5)",
    "((4+4) + (4+4))",
    "(4/4+4)",
    "(2 2+ 2)"
]

for expr in test_cases:
    print(f"{expr}: {'Valid' if is_valid_expression(expr) else 'Invalid'}")


[[]]
[['1']]
[['1', '+']]
[['1', '+', '2']]
[]
(1+2): Valid
[[]]
[['1']]
[['1']]
[['1', '+']]
[['1', '+']]
[['1', '+'], []]
[['1', '+'], ['1']]
[['1', '+'], ['1']]
[['1', '+'], ['1', '+']]
[['1', '+'], ['1', '+']]
[['1', '+'], ['1', '+', '2']]
[['1', '+', 'N']]
[]
(1 + (1 + 2)): Valid
[[]]
[['1']]
[['1']]
[['1', '+']]
[['1', '+']]
[['1', '+', '2']]
[['1', '+', '2']]
[['1', '+', '2', '+']]
[['1', '+', '2', '+']]
[['1', '+', '2', '+', '3']]
(1 + 2 + 3): Invalid
[[]]
[['1']]
[['1']]
[['1', '+']]
[['1', '+']]
[['1', '+'], []]
[['1', '+'], ['1']]
(1 + (1)): Invalid
[[]]
[['1']]
[['1']]
[['1', '+']]
[['1', '+']]
[['1', '+'], []]
[['1', '+'], ['2']]
[['1', '+'], ['2']]
[['1', '+'], ['2', '*']]
[['1', '+'], ['2', '*']]
[['1', '+'], ['2', '*', '3']]
[['1', '+', 'N']]
[]
(1 + (2 * 3)): Valid
[[]]
[[], []]
[[], ['1']]
[[], ['1']]
[[], ['1', '+']]
[[], ['1', '+']]
[[], ['1', '+', '2']]
[['N']]
[['N']]
[['N', '*']]
[['N', '*']]
[['N', '*', '3']]
[]
((1 + 2) * 3): Valid
[[]]
[['1']]
[['1']]
[['1', '

In [59]:
for exp in IVbrackets:
    print(f"{exp}: {'Valid' if is_valid_expression(exp) else 'Invalid'}")

[[]]
[['5']]
['(', '5', '**', '2', ')', '-', '1']: Invalid
[[]]
[['6']]
[['6', '+']]
[['6', '+'], []]
[['6', '+'], ['2']]
[['6', '+'], ['2', '*']]
[['6', '+'], ['2', '*', '3']]
[['6', '+', 'N']]
[]
['(', '6', '+', '(', '2', '*', '3', ')', ')', '/', '0']: Invalid
[[]]
[['4']]
[['4', '-']]
[['4', '-', '2']]
['(', '4', '-', '2', '**', '3', ')']: Invalid
[[]]
[[], []]
[[], ['9']]
[[], ['9', '-']]
[[], ['9', '-'], []]
[[], ['9', '-'], ['6']]
[[], ['9', '-'], ['6', '/']]
[[], ['9', '-'], ['6', '/', '0']]
[[], ['9', '-', 'N']]
[['N']]
['(', '(', '9', '-', '(', '6', '/', '0', ')', ')', ')']: Invalid
[[]]
[[], []]
[[], ['3']]
[[], ['3', '*']]
[[], ['3', '*', '4']]
[['N']]
[['N', '+']]
[['N', '+', '5']]
[['N', '+', '5', '/']]
[['N', '+', '5', '/'], []]
[['N', '+', '5', '/'], ['2']]
[['N', '+', '5', '/'], ['2', '-']]
[['N', '+', '5', '/'], ['2', '-', '1']]
[['N', '+', '5', '/', 'N']]
['(', '(', '3', '*', '4', ')', '+', '5', '/', '(', '2', '-', '1', ')', ')']: Invalid
[[]]
[['3']]
[['3', '+']]
[['

<h1>Validation for invalid double operators</h1>

In [60]:
IVdouble = [
    '(4+*5)',
    '(4/*5)',
    '(4***5)',
    '(4****-5)',
    '(4-*-5)'
]
Vdouble = [
    '(-1--1)',
    '(4--4)',
    '(3/-2)',
    '(2**2)',
    '(-2**-2)',
    '(-2*+2)'
]

In [61]:
def invalid_double_operators(tokens):

    for i in range(len(tokens)):
        try:
            tokens[i] = float(tokens[i])
        except:
            tokens[i] = None

    #Even indexes should be floats
    for i in range(1, len(tokens), 2):
        if tokens[i] == None:
            return True
    #Odd indexes should be None
    for i in range(0, len(tokens), 2):
        if tokens[i] != None:
            return True
    return False

invalid_double_operators(tokenize_equation(Vdouble[2]))

False

In [62]:
for exp in IVdouble:
    tokens = tokenize_equation(exp)
    print(tokens)

['(', '4', '+', '*', '5', ')']
['(', '4', '/', '*', '5', ')']
['(', '4', '**', '*', '5', ')']
['(', '4', '**', '**', '-5', ')']
['(', '4', '-', '*', '-5', ')']


<h1>Validation for syntax errors</h1>

In [63]:
IVsyntax = [
    '(-1-+-1)',
    '(/4--4)',
    '(3/-2*)',
    '(**2)',
    '(-2**)',
    '(**)'
]

In [64]:
for exp in Vdouble:
    tokens = tokenize_equation(exp)
    print("Invalid" if invalid_double_operators(tokens) else "Valid")

Valid
Valid
Valid
Valid
Valid
Valid


In [65]:
for exp in IVdouble:
    tokens = tokenize_equation(exp)
    print("Invalid" if invalid_double_operators(tokens) else "Valid")

Invalid
Invalid
Invalid
Invalid
Invalid


In [66]:
for exp in IVsyntax:
    tokens = tokenize_equation(exp)
    print("Invalid" if invalid_double_operators(tokens) else "Valid")

Invalid
Invalid
Invalid
Invalid
Invalid
Invalid


<h1>Validation for zero division (Done in evaluation)</h1>

In [72]:
for expr in IVsyntax:
    print(f"{expr}: {'Valid' if is_valid_expression(expr) else 'Invalid'}")

[[]]
[['-1']]
[['-1', '-']]
[['-1', '-', '+']]
[['-1', '-', '+', '-1']]
(-1-+-1): Invalid
[[]]
[['/']]
[['/', '4']]
[['/', '4', '-']]
[['/', '4', '-', '-4']]
(/4--4): Invalid
[[]]
[['3']]
[['3', '/']]
[['3', '/', '-2']]
[['3', '/', '-2', '*']]
(3/-2*): Invalid
[[]]
[['**']]
[['**', '2']]
(**2): Invalid
[[]]
[['-2']]
[['-2', '**']]
(-2**): Invalid
[[]]
[['**']]
(**): Invalid
